# Groupwise cortical parcellation

In [15]:
"""
||AUM||
||Shree Ganeshaya Namaha||

"""
%reset
from dfsio import readdfs,writedfs
import scipy.io
import numpy as np
import csv
from dfsio import readdfs,writedfs
from mayavi import mlab
import h5py
import os
p_dir = 'C:\\Users\\ajoshi\\Downloads\\HCP-fMRI-NLM';
sub = '100307';
r_factor = 3;
ref_dir = os.path.join(p_dir, 'reference');
ref = '100307';
print(ref + '.reduce' + str(r_factor) + '.LR_mask.mat')
fn1=ref + '.reduce' + str(r_factor) + '.LR_mask.mat'
fname1=os.path.join(ref_dir,fn1 )
msk = scipy.io.loadmat(fname1) #h5py.File(fname1);
data = scipy.io.loadmat(os.path.join(p_dir, sub, sub + '.rfMRI_REST1_RL.reduce3.ftdata.NLM_11N_hvar_25.mat'));

dfs_left = readdfs(os.path.join(p_dir, 'reference', sub + '.aparc.a2009s.32k_fs.reduce3.left.dfs'));
dfs_left_sm = readdfs(os.path.join(p_dir, 'reference', sub + '.aparc.a2009s.32k_fs.reduce3.very_smooth.left.dfs'));

LR_flag=msk['LR_flag']
LR_flag=np.squeeze(LR_flag)>0
data=data['ftdata_NLM']
temp = data[LR_flag, :]

msk_small_region = (dfs_left.labels==46) | (dfs_left.labels==28) | (dfs_left.labels==29) #% motor

#temp = data.ftdata_NLM;%(msk.LR_flag, :);
d = temp[msk_small_region , :];
d_corr = temp[~msk_small_region , :];

#% make each signal mean-0 and variance-1
m = np.mean(d, 1);
d = d - m[:,None]#(:, np.ones((1, size(d, 2)]);
s = np.std(d, 1);
d = d/s[:,None]
#g_mask = np.any(~np.isfinite(d),1);
#d[g_mask, :] = 0;


#% compute correlation matrix
rho = np.corrcoef(d, d_corr);
rho=rho[range(d.shape[0]),d.shape[0]+1:]
rho[~np.isfinite(rho)] = 0;
rho_rho = np.corrcoef(rho);

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
100307.reduce3.LR_mask.mat


In [9]:
import os
from dfsio import readdfs,writedfs
import networkx as nx
g = nx.Graph()
p_dir = 'C:\\Users\\ajoshi\\Downloads\\HCP-fMRI-NLM';
sub = '100307';
s = readdfs(os.path.join(p_dir, 'reference', sub + '.aparc.a2009s.32k_fs.reduce3.very_smooth.left.dfs'));
g.add_edges_from(s.faces[:,(0,1)]);g.add_edges_from(s.faces[:,(1,2)]);g.add_edges_from(s.faces[:,(2,0)]);
Adj=nx.adjacency_matrix(g)

In [10]:
AdjS=Adj[(msk_small_region),:];AdjS=AdjS[:,(msk_small_region)]
AdjS=AdjS.todense()
#AdjS=np.matrix(AdjS)
np.fill_diagonal(AdjS,1)
#AdjS.shape
AdjS=1.0*((AdjS*AdjS*AdjS)>0)
rho_rho=np.multiply(AdjS,rho_rho)
np.sum(AdjS,1)
type(AdjS)

numpy.matrixlib.defmatrix.matrix

In [ ]:
from sklearn.covariance import GraphLassoCV
dd=np.concatenate((d,d_corr),0)
model = GraphLassoCV(mode='lars')
model.fit(dd) 
rho2 = model.covariance_
rho2=rho2[range(d.shape[0]),d.shape[0]+1:]
rho2[~np.isfinite(rho2)] = 0;
model.fit(rho2)
rho_rho2=model.covariance_

In [1]:
rho = model.covariance_
rho=rho[range(d.shape[0]),d.shape[0]+1:]
rho[~np.isfinite(rho)] = 0;
model.fit(rho)
rho_rho=model.covariance_

NameError: name 'model' is not defined

In [11]:
np.fill_diagonal(rho_rho,0)

In [12]:
#from skimage import data, io, segmentation, color
#from skimage.future import graph
from sklearn.cluster import SpectralClustering

In [13]:
AdjS

matrix([[ 1.,  1.,  1., ...,  0.,  0.,  0.],
        [ 1.,  1.,  1., ...,  0.,  0.,  0.],
        [ 1.,  1.,  1., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  1.,  1.,  1.],
        [ 0.,  0.,  0., ...,  1.,  1.,  1.],
        [ 0.,  0.,  0., ...,  1.,  1.,  1.]])

In [14]:
for nclusters in [2, 5, 10, 30,200]:
    SC=SpectralClustering(n_clusters=nclusters,affinity='precomputed')
    labs=SC.fit_predict(rho_rho)

    r=dfs_left_sm;r.labels=r.labels*0;r.labels[msk_small_region]=labs+1;
    mesh = mlab.triangular_mesh(r.vertices[:,0], r.vertices[:,1], r.vertices[:,2], r.faces, representation='surface', 
                            opacity=1,scalars=np.float64(r.labels))
    #mlab.pipeline.surface(mesh)

    mlab.gcf().scene.parallel_projection = True
    mlab.view(azimuth=0, elevation=-90)
    mlab.savefig(filename = str(nclusters)+'labels1.png')
    mlab.view(azimuth=0, elevation=90)
    mlab.savefig(filename = str(nclusters)+'labels2.png')
    mlab.close();

In [6]:
str(5)

'5'

In [7]:
np.min(labs)

0